In [1]:
import os

os.chdir('/opt/ml/P-Stage/1-STAGE/')

from PIL import Image

import wandb
import torch
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F

import matplotlib.pyplot as plt
from sklearn.metrics import (
    f1_score,
    recall_score,
    accuracy_score,
    precision_score,
    confusion_matrix,

)

from config import get_args
from prepare import get_dataloader, get_transforms, get_classes
from predict import load_models
from metrics import (
    FocalLoss,
    change_2d_to_1d,
    calulate_18class,
    tensor_to_numpy,
    tensor_images_to_numpy_images,
)

args = get_args()
args.device = "cuda:0" if torch.cuda.is_available() else "cpu"
args.test = False
args.use_only_mask = True
args.train_key = "mask"

train_dataloader, valid_dataloader = get_dataloader(args)

In [3]:
models = load_models(args)

In [7]:
images, labels = next(iter(train_dataloader))
images, labels = images.cuda(), labels.cuda()

In [5]:
class FocalLoss(nn.Module):
    def __init__(self, weight=None, gamma=2.0, reduction="mean"):
        nn.Module.__init__(self)
        self.weight = weight
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, input_tensor, target_tensor):
        log_prob = F.log_softmax(input_tensor, dim=-1)
        prob = torch.exp(log_prob)
        return F.nll_loss(
            ((1 - prob) ** self.gamma) * log_prob,
            target_tensor,
            weight=self.weight,
            reduction=self.reduction,
        )


class LabelSmoothingLoss(nn.Module):
    def __init__(self, classes=3, smoothing=0.0, dim=-1):
        super(LabelSmoothingLoss, self).__init__()
        self.confidence = 1.0 - smoothing
        self.smoothing = smoothing
        self.cls = classes
        self.dim = dim

    def forward(self, pred, target):
        pred = pred.log_softmax(dim=self.dim)
        with torch.no_grad():
            true_dist = torch.zeros_like(pred)
            true_dist.fill_(self.smoothing / (self.cls - 1))
            true_dist.scatter_(1, target.data.unsqueeze(1), self.confidence)
        return torch.mean(torch.sum(-true_dist * pred, dim=self.dim))


# https://gist.github.com/SuperShinyEyes/dcc68a08ff8b615442e3bc6a9b55a354
class F1Loss(nn.Module):
    def __init__(self, classes=3, epsilon=1e-7):
        super().__init__()
        self.classes = classes
        self.epsilon = epsilon

    def forward(self, y_pred, y_true):
        assert y_pred.ndim == 2
        assert y_true.ndim == 1

        y_true = F.one_hot(y_true, self.classes).to(torch.float32)
        y_pred = F.softmax(y_pred, dim=1)

        tp = (y_true * y_pred).sum(dim=0).to(torch.float32)
        tn = ((1 - y_true) * (1 - y_pred)).sum(dim=0).to(torch.float32)
        fp = ((1 - y_true) * y_pred).sum(dim=0).to(torch.float32)
        fn = (y_true * (1 - y_pred)).sum(dim=0).to(torch.float32)

        precision = tp / (tp + fp + self.epsilon)
        recall = tp / (tp + fn + self.epsilon)

        f1 = 2 * (precision * recall) / (precision + recall + self.epsilon)
        f1 = f1.clamp(min=self.epsilon, max=1 - self.epsilon)
        return 1 - f1.mean()

In [8]:
preds = models[0](images)
preds.shape

torch.Size([64, 3])

In [13]:
f1_loss = F1Loss()
focal_loss = FocalLoss()
smoothing_loss = LabelSmoothingLoss()

In [14]:
f1_loss(preds, labels)

tensor(0.5093, device='cuda:0', grad_fn=<RsubBackward1>)

In [15]:
focal_loss(preds, labels)

tensor(0.4115, device='cuda:0', grad_fn=<MeanBackward0>)

In [16]:
smoothing_loss(preds, labels)

tensor(0.4115, device='cuda:0', grad_fn=<MeanBackward0>)